# Lorentz matrix doodles

Idea: see what it takes to do things "just like a Lorentz matrix."

In [ ]:
%%capture
%matplotlib inline
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
import math
import unittest
from copy import deepcopy
import pudb

# To get equations the look like, well, equations, use the following.
from sympy.interactive import printing
printing.init_printing(use_latex=True)
from IPython.display import display

# Tools for manipulating quaternions written by D. Sweetser.
from QH import QH, QHStates

from IPython.core.display import display, HTML, Math, Latex
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
t, x, y, z = sp.symbols("t x y z")

In [ ]:
v_txyz = sp.Matrix([t,x,y,z])

In [ ]:
Lyx = sp.Matrix([[1, 0, 0, 0],
                 [0, 0, -1, 0],
                 [0, 1, 0, 0],
                 [0, 0, 0, 1]])
display(Lyx)
display(Lyx * v_txyz)

Notice that each term is algebraically isolated from the others. To do this with quaternions, the same thing must be done - terms must be isolated from each other. This can be done using the three conjugates and a function to flip signs. For t, one uses the standard conjugates. To isolate x and y needs the action of the first and second conjugates. TO get z, all three conjugates and the sign flip.

In [ ]:
q_txyz = QH([t, x, y, z])

In [ ]:
q_txyz.add(q_txyz.conj()).print_state("2 t")
q_txyz.add(q_txyz.conj(1)).print_state("2 x")
q_txyz.add(q_txyz.conj(2)).print_state("2 y")
q_txyz.add(q_txyz.conj().conj(1).conj(2).flip_signs()).print_state("2 z")

Now twist the $x$ and $y$ states with a product with $k$.

In [ ]:
q_txyz.add(q_txyz.conj(1)).product(QH().q_k(-1)).print_state("2 x")
q_txyz.add(q_txyz.conj(2)).product(QH().q_k(-1)).print_state("2 y")

Just add it all up.

In [ ]:
q_txyz.add(q_txyz.conj()).add(q_txyz.add(q_txyz.conj(1)).product(QH().q_k(-1))).add(q_txyz.add(q_txyz.conj(2)).product(QH().q_k(-1))).add(q_txyz.add(q_txyz.conj().conj(1).conj(2).flip_signs())).product(QH().q_1(1/2)).display_q("tyxz")

In [ ]:
q_i = QH().q_i()
q_i_exp = q_i.exp()
q_i_exp.print_state("e^i")

In [ ]:
print("arccosh: ", np.arccosh(q_i_exp.t))
print("arcsinh: ", np.arcsinh(q_i_exp.x))

In [ ]:
print("arccosh: ", np.cosh(q_i_exp.t))
print("arcsinh: ", np.sinh(q_i_exp.x))

In [ ]:
q_i_exp.norm_squared().print_state("|e^i|^2")

In [ ]:
qv = QH([0, 3, 1, 2]).normalize()
qv.print_state("qv")

In [ ]:
q_txyz_qv = q_txyz.boost_or_rotation(qv)
q_txyz_qv.print_state("q_txyz_qv")

In [ ]:
qb_2 = q_txyz_qv.product(q_txyz_qv).simple_q()
qb_2.print_state("qb_2")

In [ ]:
np.set_printoptions(suppress=True)
q_txyz_qv_exp = q_txyz.boost_or_rotation(qv.exp().normalize(np.sqrt(1/0.415008189568194)))
q_txyz_qv_exp.print_state("q_txyz_qv_exp")
qb_2_exp = q_txyz_qv_exp.product(q_txyz_qv_exp).simple_q()
qb_2_exp.print_state("qb_exp_2")

Demonstrate numerically that the vector normalized cross product does a rotation.

In [ ]:
q1234_n = QH([1, 2, 3, 4]).vector().normalize()
q4321_n = QH([4, 3, -2, 1]).vector().normalize()

In [ ]:
q1234_n.print_state("q1234_n")
q1234_n.norm_squared().print_state("|q1234_n|^2")
q4321_n.print_state("q4321_n")
q4321_n.norm_squared().print_state("|q4321_n|^2")

In [ ]:
q_234x432_n = q1234_n.product(q4321_n, kind="odd").vector().normalize()

In [ ]:
q_2244 = QH([2, 2, 4, 4])
q_2244_sq = q_2244.square()
q_2244_sq.print_state("q_2244 sq")

In [ ]:
q_2244_b_q_1234_n = q_2244.boost_or_rotation(q1234_n)
q_2244_b_q_1234_n.print_state("q_2244_b_q_1234_n")
q_2244_b_q_1234_n.square().print_state("q_2244_b_q_1234_n sq")

q_2244_b_q_4321_n = q_2244.boost_or_rotation(q4321_n)
q_2244_b_q_4321_n.print_state("q_2244_b_q_4321_n")
q_2244_b_q_4321_n.square().print_state("q_2244_b_q_4321_n sq")

q_2244_b_q_234x432_n = q_2244.boost_or_rotation(q_234x432_n)
q_2244_b_q_234x432_n.print_state("q_2244_b_q_234x432_n")
q_2244_b_q_234x432_n.square().print_state("q_2244_b_q_234x432_n sq")

These three rotations all work as expected. The first term is fixed at the value 2.0, plus or minus some numerical error. The directions pointed to are all over the map. Yet the first term of the square is always -32.0 with numerical error.

Now see if boosts can be done...

In [ ]:
q_2244_b_q_1234_n_exp = q_2244.boost_or_rotation(q1234_n.exp())
q_2244_b_q_1234_n_exp.print_state("q_2244_b_q_1234_exp_n")
q_2244_b_q_1234_n_exp.square().print_state("q_2244_b_q_1234_exp_n sq")

In [ ]:
q_2244_b_q_1234_n_exp_n = q_2244.boost_or_rotation(q1234_n.exp().normalize())
q_2244_b_q_1234_n_exp_n.print_state("q_2244_b_q_1234_exp_n")
q_2244_b_q_1234_n_exp_n.square().print_state("q_2244_b_q_1234_exp_n sq")

Not even close. Need to do something so 1st term > 1.0.

In [ ]:
q1234_n.exp().print_state("q1234_n_exp")
q1234_n.exp().norm_squared().print_state("|q123_or_rotation4_n_exp|^2")

In [ ]:
q1234_n.exp().print_state("q1234_n_exp")
q1234_n.ln().print_state("q1234_n_ln")
q1234_n.cos().print_state("q1234_n_cos")
q1234_n.sin().print_state("q1234_n_sin")
q1234_n.cosh().print_state("q1234_n_cosh")
q1234_n.sinh().print_state("q1234_n_sinh")
#q1234_n.acos().print_state("q1234_n_acos")
#q1234_n.asin().print_state("q1234_n_asin")
#q1234_n.acosh().print_state("q1234_n_acosh")
#q1234_n.asinh().print_state("q1234_n_asinh")

In [ ]:
q_1234_n_asin = QH([0.000000, 0.327334, 0.491001, 0.654668])
q_1234_n_acos = QH([1.190426, -0.259756, -1.209502, 0.095389])
q_1234_n_asinh = QH([0.000000, 0.583379, 0.875069, 1.166758])
q_1234_n_acosh = QH([0.699414, 1.393649, 0.434801, 0.579735])

In [ ]:
q_2244_b_q_1234_acos = q_2244.boost_or_rotation(q_1234_n_acos)
q_2244_b_q_1234_acos.print_state("q_2244_b_q_1234_acos")
q_2244_b_q_1234_acos.square().print_state("q_2244_b_q_1234_acos sq")

In [ ]:
q_2244_b_q_1234_acos_n = q_2244.boost_or_rotation(q_1234_n_acos.normalize())
q_2244_b_q_1234_acos_n.print_state("q_2244_b_q_1234_acos_n")
q_2244_b_q_1234_acos_n.square().print_state("q_2244_b_q_1234_acos_n sq")

In [ ]:
q_2244_b_cs = q_2244.boost_or_rotation(QH([np.cosh(2), np.sinh(2),0,0]))
q_2244_b_cs.print_state("q_2244_b_cs_n")
q_2244_b_cs.square().print_state("q_2244_b_cs sq")

In [ ]:
q_2244_b_cs = q_2244.boost_or_rotation(QH([0.699414, 0.583379, 0.875069, 1.166758]))
q_2244_b_cs.print_state("q_2244_b_cs_n")
q_2244_b_cs.square().print_state("q_2244_b_cs sq")

In [ ]:
q_2244_b_cs = q_2244.boost_or_rotation(QH([1.190426, 0.3713906763541038, 0.5570860145311557, 0.7427813527082]))
q_2244_b_cs.print_state("q_2244_b_cs_n")
q_2244_b_cs.square().print_state("q_2244_b_cs sq")

In [ ]:
q_2244_b_cs = q_2244.boost_or_rotation(QH([1.190426, 0.583379, 0.875069, 1.166758]))
q_2244_b_cs.print_state("q_2244_b_cs_n")
q_2244_b_cs.square().print_state("q_2244_b_cs sq")

Try $(\exp(a) + \exp(-a))/2$, $(\exp(a) - \exp(-a))/2$ 

In [ ]:
exp_a = q1234_n.exp()
exp_ac = q1234_n.conj().exp()
exp_sum = exp_a.add(exp_ac).product(QH([1, 0, 0, 0]))
exp_dif = exp_a.dif(exp_ac).product(QH([1, 0, 0, 0]))
exp_sum.print_state("exp sum")
exp_dif.print_state("exp_dif")

In [ ]:
exp_a.print_state("exp_a")
exp_ac.print_state("exp_ac")
exp_sum.print_state("exp_sum")
exp_dif.print_state("exp_dif")

In [ ]:
q_2244_b_exp_sum_dif = q_2244.boost_or_rotation(exp_sum.add(exp_dif))
q_2244_b_exp_sum_dif.print_state("q_2244_b_exp_sum_dif")
q_2244_b_exp_sum_dif.square().print_state("q_2244_b_exp_sum_dif")

In [ ]:
q1234_n.print_state("q_1234_n")

Take each term separately and invert it.

In [ ]:
1/0.3713906763541038
1/0.5570860145311557
1/0.7427813527082076

To calculate the hyperbolic cosine, one needs the sum/even part of an exponential of a real number:

$$\cosh(x) = \frac{e^{x}+e^{-x}}{2}$$

whereas sinh(x) is the difference:

$$\sinh(x) = \frac{e^{x}-e^{-x}}{2}$$

In [ ]:
xsum=(np.exp(2.692582403567252)+np.exp(-2.692582403567252))/2
xdif=(np.exp(2.692582403567252)-np.exp(-2.692582403567252))/2
print("xsum: ", xsum)
print("xdif: ", xdif)

Use the sum and the difference to create a boost quaternion, one that is a combination of a xsum and i xdif.

In [ ]:
qboost = QH([xsum, 0, 0, 0]).add(q1234_n.product(QH([xdif,0,0,0])))
qboost.print_state("qboost")

In [ ]:
q_2244_b_qboost = q_2244.boost_or_rotation(qboost)
q_2244_b_qboost.print_state("q_2244_b_qboost")
q_2244_b_qboost.square().print_state("q_2244_b_q_boost")

In [ ]:
q1234_n.add(q1234_n.conj(1)).print_state("q1234_n + *1")
q1234_n.add(q1234_n.conj(1)).inverse().print_state("q1234_n + *1 inv")
q1234_n.add(q1234_n.conj(1)).inverse().product(QH().q_i()).print_state("q1234_n + *1 inv i")

In [ ]:
QH().q_0().add(QH().q_0().conj(1)).inverse().print_state("0 + *1 inv")

In [ ]:
def Lorentz_next_rotation(q1, q2):
    """Given 2 quaternions, creates a new quaternion to do a rotation
       in the triple triple quaternion function by using a normalized cross product."""
    
    next_rotation = q1.product(q2, kind="odd").normalize()
    
    # If the 2 quaternions point in exactly the same direction, the result is zoro.
    # That is unacceptable for closure, so return the normalized vector of one input.
    # This does create some ambiguity since q1 and q2 could point in exactly opposite
    # directions. In that case, the first quaternion is always chosen.
    v_norm = next_rotation.norm_squared_of_vector()
    
    if v_norm.t == 0:
        next_rotation = q1.vector().normalize()

    return next_rotation

def Lorentz_next_boost(q1, q2):
    """Given 2 quaternions, creates a new quaternion to do a boost/rotation
       using the triple triple quaternion product
       by using the scalar of an even product to form (cosh(x), i sinh(x))."""
    
    q_even = q1.product(q2, kind="even")
    q_s = q_even.scalar()
    q_v = q_even.vector().normalize()
    
    if np.abs(q_s.t) > 1:
        q_s = q_s.inverse()
    
    exp_sum = q_s.exp().add(q_s.flip_signs().exp()).product(QH().q_1(1/2))
    exp_dif = q_s.exp().dif(q_s.flip_signs().exp()).product(QH().q_1(1/2))
    
    boost = exp_sum.add(q_v.product(exp_dif))
    
    return boost

if __name__ == "__main__":
    class TestLorentz(unittest.TestCase):
    
        Q = QH([1, -2, -3, -4], qtype="Q")
        P = QH([0, 4, -3, 0], qtype="P")
        R = QH([3, 0, 0, 0], qtype="R")
        C = QH([2, 4, 0, 0], qtype="C")
        q4321 = QH([4, 3, 2, 1])
        q2244 = QH([2, 2, 4, 4])
        
        t, x, y, z = sp.symbols("t x y z")
        q_sym = QH([t, x, y, x * y * z])
        
        def test_Lorentz_next_rotation(self):
            next_rotation = Lorentz_next_rotation(self.Q, self.q4321)
            print("next_rotation: ", next_rotation)
            self.assertEqual(next_rotation.t, 0)
            rot = self.q2244.boost_or_rotation(next_rotation)
            self.assertEqual(rot.t, 2)
            self.assertAlmostEqual(rot.square().t, self.q2244.square().t)
            next_rotation = Lorentz_next_rotation(self.Q, self.Q)
            self.assertTrue(next_rotation.equals(self.Q.vector().normalize()))
            
            
        def test_Lorentz_next_boost(self):
            next_boost = Lorentz_next_boost(self.Q, self.q4321)
            print("next_boost: ", next_boost)
            self.assertNotEqual(next_boost.t, 0)
            boost = self.q2244.boost_or_rotation(next_boost)
            self.assertAlmostEqual(boost.square().t, self.q2244.square().t)
            
    suite = unittest.TestLoader().loadTestsFromModule(TestLorentz())
    _results = unittest.TextTestRunner().run(suite); 

qnext = Lorentz_next_rotation(QH([1, 2, 3, 4]), QH([4, 3, 2, 5]))
qnext.print_state("qnext")
q_2244_b_qnext = q_2244.boost_or_rotation(qnext)
q_2244_b_qnext.print_state("q_2244_b_qnext")
q_2244_b_qnext.square().print_state("q_2244_b_qnext")

In [ ]:
qnext = Lorentz_next_rotation(QH([1, 2, 3, 4]), QH([4, 2, 3, 4]))
qnext.print_state("qnext")
q_2244_b_qnext = q_2244.boost_or_rotation(qnext)
q_2244_b_qnext.print_state("q_2244_b_qnext")
q_2244_b_qnext.square().print_state("q_2244_b_qnext")

In [ ]:
qnext = Lorentz_next_boost(QH([1, 2, 3, 4]), QH([4, 3, 2, 1]))
qnext.print_state("qnext boost")
q_2244_b_qnext = q_2244.boost_or_rotation(qnext)
q_2244_b_qnext.print_state("q_2244_b_qnext")
q_2244_b_qnext.square().print_state("q_2244_b_qnext sq")

In [ ]:
QH().q_i().product(QH().q_j(), kind="even").print_state("ij")

In [ ]:
def Lorentz_next(q1, q2):
    """Do both since both work, combine at the end."""
    
    rotation = Lorentz_next_rotation(q1, q2)
    rotation.print_state("rotation is:")
    boost = Lorentz_next_boost(q1, q2)
    boost.print_state("boost is:")
    rotation_boost = boost.product(rotation).normalize()
    
    return rotation_boost

qnext = Lorentz_next(QH([1, 2, 3, 4]), QH([4, 3, 2, 1]))
qnext.print_state("qnext rotation & boost")
q_2244_b_qnext = q_2244.boost_or_rotation(qnext)
q_2244_b_qnext.print_state("q_2244_b_qnext")
q_2244_b_qnext.square().print_state("q_2244_b_qnext sq")

In [ ]:
qnext = Lorentz_next_boost(QH([.1, .2, .03, .04]), QH([.04, .3, .2, .1]))
qnext.print_state("qnext boost")
q_2244_b_qnext = q_2244.boost_or_rotation(qnext)
q_2244_b_qnext.print_state("q_2244_b_qnext")
q_2244_b_qnext.square().print_state("q_2244_b_qnext sq")

In [ ]:
qnext = Lorentz_next_boost(QH([.1, .2, .03, .04]), QH([.04, .3, .2, .1]))
qnext.print_state("qnext boost")
q_2244_b_qnext = q_2244.boost_or_rotation(qnext)
q_2244_b_qnext.print_state("q_2244_b_qnext")
q_2244_b_qnext.square().print_state("q_2244_b_qnext sq")

In [ ]:
def Lorentz_next_rotationQS(self, q1):
    """Does multiple rotations of a QHState given another QHState of equal dimensions."""
    
    if self.dim != q1.dim:
        print("Oops, this tool requires 2 quaternion states with the same number of dimensions.")
        return null
    
    new_states = []
        
    for ket, q in zip(self.qs, q1.qs):
        new_states.append(Lorentz_next_rotation(ket, q))
            
    return QHStates(new_states, qs_type=self.qs_type, rows=self.rows, columns=self.columns)
   

def Lorentz_next_boostQS(self, q1):
    """Does multiple boosts of a QHState given another QHState of equal dimensions."""
          
    if self.dim != q1.dim:
        print("Oops, this tool requires 2 quaternion states with the same number of dimensions.")
        return null
                      
    new_states = []
        
    for ket, q in zip(self.qs, q1.qs):
        new_states.append(Lorentz_next_boost(ket, q))
       
    return QHStates(new_states, qs_type=self.qs_type, rows=self.rows, columns=self.columns)
   

if __name__ == "__main__":
  

    class TestQHStates(unittest.TestCase):
        """Test states."""

        q_0 = QH().q_0()
        q_1 = QH().q_1()
        q_i = QH().q_i()
        q_n1 = QH([-1,0,0,0])
        q_2 = QH([2,0,0,0])
        q_n2 = QH([-2,0,0,0])
        q_3 = QH([3,0,0,0])
        q_n3 = QH([-3,0,0,0])
        q_4 = QH([4,0,0,0])
        q_5 = QH([5,0,0,0])
        q_6 = QH([6,0,0,0])
        q_10 = QH([10,0,0,0])
        q_n5 = QH([-5,0,0,0])
        q_7 = QH([7,0,0,0])
        q_8 = QH([8,0,0,0])
        q_9 = QH([9,0,0,0])
        q_n11 = QH([-11,0,0,0])
        q_21 = QH([21,0,0,0])
        q_n34 = QH([-34,0,0,0])
        v3 = QHStates([q_3])
        v1123 = QHStates([q_1, q_1, q_2, q_3])
        v3n1n21 = QHStates([q_3,q_n1,q_n2,q_1])
        v9 = QHStates([q_1, q_1, q_2, q_3, q_1, q_1, q_2, q_3, q_2])
        v9i = QHStates([QH([0,1,0,0]), QH([0,2,0,0]), QH([0,3,0,0]), QH([0,4,0,0]), QH([0,5,0,0]), QH([0,6,0,0]), QH([0,7,0,0]), QH([0,8,0,0]), QH([0,9,0,0])])
        vv9 = v9.add(v9i)
        q_1d0 = QH([1.0, 0, 0, 0])
        q12 = QHStates([q_1d0, q_1d0])
        q14 = QHStates([q_1d0, q_1d0, q_1d0, q_1d0])
        q19 = QHStates([q_1d0, q_0, q_1d0, q_1d0, q_1d0, q_1d0, q_1d0, q_1d0, q_1d0])
        qn627 = QH([-6,27,0,0])
        v33 = QHStates([q_7, q_0, q_n3, q_2, q_3, q_4, q_1, q_n1, q_n2])
        v33inv = QHStates([q_n2, q_3, q_9, q_8, q_n11, q_n34, q_n5, q_7, q_21])
        q_i3 = QHStates([q_1, q_1, q_1])
        q_i2d = QHStates([q_1, q_0, q_0, q_1])
        q_i3_bra = QHStates([q_1, q_1, q_1], "bra")
        q_6_op = QHStates([q_1, q_0, q_0, q_1, q_i, q_i], "op")    
        q_6_op_32 = QHStates([q_1, q_0, q_0, q_1, q_i, q_i], "op", rows=3, columns=2)
        q_i2d_op = QHStates([q_1, q_0, q_0, q_1], "op")
        q_i4 = QH([0,4,0,0])
        q_0_q_1 = QHStates([q_0, q_1])
        q_1_q_0 = QHStates([q_1, q_0])
        q_1_q_i = QHStates([q_1, q_i])
        q_1_q_0 = QHStates([q_1, q_0])
        q_0_q_i = QHStates([q_0, q_i])
        A = QHStates([QH([4,0,0,0]), QH([0,1,0,0])], "bra")
        B = QHStates([QH([0,0,1,0]), QH([0,0,0,2]), QH([0,3,0,0])])
        Op = QHStates([QH([3,0,0,0]), QH([0,1,0,0]), QH([0,0,2,0]), QH([0,0,0,3]), QH([2,0,0,0]), QH([0,4,0,0])], "op", rows=2, columns=3)
        Op4i = QHStates([q_i4, q_0, q_0, q_i4, q_2, q_3], "op", rows=2, columns=3) 
        Op_scalar = QHStates([q_i4], "scalar")
        q_1234 = QHStates([QH([1, 1, 0, 0]), QH([2, 1, 0, 0]), QH([3, 1, 0, 0]), QH([4, 1, 0, 0])])
        sigma_y = QHStates([QH([1, 0, 0, 0]), QH([0, -1, 0, 0]), QH([0, 1, 0, 0]), QH([-1, 0, 0, 0])])
        qn = QHStates([QH([3,0,0,4])])
        q_bad = QHStates([q_1], rows=2, columns=3)

        b = QHStates([q_1, q_2, q_3], qs_type="bra")
        k = QHStates([q_4, q_5, q_6], qs_type="ket")
        o = QHStates([q_10], qs_type="op")
        
        q1234 = QH([1, 2, 3, 4])
        q4321 = QH([4, 3, 2, 1])
        q2222 = QH([2, 2, 2, 2])
        qsmall = QH([.04, 0.2, 0.1, -0.3])
        q2_states = QHStates([q1234, qsmall], "ket")
        
        def test_Lorentz_next_rotation(self):
            next_rot = Lorentz_next_rotationQS(self.q2_states, QHStates([self.q2222, self.q2222]))
            print("next_rotation: ", next_rot)
            self.assertEqual(next_rot.qs[0].t, 0)
            self.assertEqual(next_rot.qs[1].t, 0)
            self.assertAlmostEqual(next_rot.norm_squared().qs[0].t, 2)
            self.assertFalse(next_rot.qs[0].equals(next_rot.qs[1]))
            
        def test_Lorentz_next_boost(self):
            next_boost = Lorentz_next_boostQS(self.q2_states, QHStates([self.q2222, self.q2222]))
            print("next_boost: ", next_boost)
            self.assertNotEqual(next_boost.qs[0].t, 0)
            self.assertNotEqual(next_boost.qs[1].t, 0)
            self.assertNotEqual(next_boost.norm_squared().qs[0].t, 2)
            self.assertFalse(next_boost.qs[0].equals(next_boost.qs[1]))
            boosted_square = self.q2_states.boost_or_rotation(next_boost).square()
            q2_states_square = self.q2_states.square()
            self.assertAlmostEqual(q2_states_square.qs[0].t, boosted_square.qs[0].t)
        

    suite = unittest.TestLoader().loadTestsFromModule(TestQHStates())
    _results = unittest.TextTestRunner().run(suite);  
    

In [ ]:
!pip install pudb

In [ ]:
q_0 = QH().q_0()
q_1 = QH().q_1()
q_i = QH().q_i()
q_n1 = QH([-1,0,0,0])
q_2 = QH([2,0,0,0])
q_n2 = QH([-2,0,0,0])
q_3 = QH([3,0,0,0])
q_n3 = QH([-3,0,0,0])
q_4 = QH([4,0,0,0])
q_5 = QH([5,0,0,0])
q_6 = QH([6,0,0,0])
q_10 = QH([10,0,0,0])
q_n5 = QH([-5,0,0,0])
q_7 = QH([7,0,0,0])
q_8 = QH([8,0,0,0])
q_9 = QH([9,0,0,0])
q_n11 = QH([-11,0,0,0])
q_21 = QH([21,0,0,0])
q_n34 = QH([-34,0,0,0])
v3 = QHStates([q_3])
v1123 = QHStates([q_1, q_1, q_2, q_3])
v3n1n21 = QHStates([q_3,q_n1,q_n2,q_1])
v9 = QHStates([q_1, q_1, q_2, q_3, q_1, q_1, q_2, q_3, q_2])
v9i = QHStates([QH([0,1,0,0]), QH([0,2,0,0]), QH([0,3,0,0]), QH([0,4,0,0]), QH([0,5,0,0]), QH([0,6,0,0]), QH([0,7,0,0]), QH([0,8,0,0]), QH([0,9,0,0])])
vv9 = v9.add(v9i)
q_1d0 = QH([1.0, 0, 0, 0])
q12 = QHStates([q_1d0, q_1d0])
q14 = QHStates([q_1d0, q_1d0, q_1d0, q_1d0])
q19 = QHStates([q_1d0, q_0, q_1d0, q_1d0, q_1d0, q_1d0, q_1d0, q_1d0, q_1d0])
qn627 = QH([-6,27,0,0])
v33 = QHStates([q_7, q_0, q_n3, q_2, q_3, q_4, q_1, q_n1, q_n2])
v33inv = QHStates([q_n2, q_3, q_9, q_8, q_n11, q_n34, q_n5, q_7, q_21])
q_i3 = QHStates([q_1, q_1, q_1])
q_i2d = QHStates([q_1, q_0, q_0, q_1])
q_i3_bra = QHStates([q_1, q_1, q_1], "bra")
q_6_op = QHStates([q_1, q_0, q_0, q_1, q_i, q_i], "op")    
q_6_op_32 = QHStates([q_1, q_0, q_0, q_1, q_i, q_i], "op", rows=3, columns=2)
q_i2d_op = QHStates([q_1, q_0, q_0, q_1], "op")
q_i4 = QH([0,4,0,0])
q_0_q_1 = QHStates([q_0, q_1])
q_1_q_0 = QHStates([q_1, q_0])
q_1_q_i = QHStates([q_1, q_i])
q_1_q_0 = QHStates([q_1, q_0])
q_0_q_i = QHStates([q_0, q_i])
A = QHStates([QH([4,0,0,0]), QH([0,1,0,0])], "bra")
B = QHStates([QH([0,0,1,0]), QH([0,0,0,2]), QH([0,3,0,0])])
Op = QHStates([QH([3,0,0,0]), QH([0,1,0,0]), QH([0,0,2,0]), QH([0,0,0,3]), QH([2,0,0,0]), QH([0,4,0,0])], "op", rows=2, columns=3)
Op4i = QHStates([q_i4, q_0, q_0, q_i4, q_2, q_3], "op", rows=2, columns=3) 
Op_scalar = QHStates([q_i4], "scalar")
q_1234 = QHStates([QH([1, 1, 0, 0]), QH([2, 1, 0, 0]), QH([3, 1, 0, 0]), QH([4, 1, 0, 0])])
sigma_y = QHStates([QH([1, 0, 0, 0]), QH([0, -1, 0, 0]), QH([0, 1, 0, 0]), QH([-1, 0, 0, 0])])
qn = QHStates([QH([3,0,0,4])])
q_bad = QHStates([q_1], rows=2, columns=3)

b = QHStates([q_1, q_2, q_3], qs_type="bra")
k = QHStates([q_4, q_5, q_6], qs_type="ket")
o = QHStates([q_10], qs_type="op")

q1234 = QH([1, 2, 3, 4])
q4321 = QH([4, 3, 2, 1])
q2222 = QH([2, 2, 2, 2])
qsmall = QH([.04, 0.2, 0.1, -0.3])
q2_states = QHStates([q1234, qsmall], "ket")

In [ ]:

next_rot = Lorentz_next_rotationQS(q2_states, q2222)
next_rot.print_state("next_rot")

In [ ]:
for ket in q2_states.qs:
    ket.print_state("qs")

In [ ]:
next_rot = Lorentz_next_boostQS(q2_states, q2222)
next_rot.print_state("next_rot")

In [ ]:
Lorentz_next_boostQS(q2_states, q2222).norm_squared().print_state("boost norm")